In [ ]:
import torch
import torch.nn.functional as F
import hydra
import torch
from lib.utils.config import load_config
from lib.optimizer.framework import NeuralOptimizer
from lib.data.loader import load_intrinsics
from lib.data.loader import load_intrinsics
from lib.rasterizer import Rasterizer
from lib.renderer.renderer import Renderer
from lib.renderer.camera import Camera
from lib.utils.visualize import visualize_point2plane_error
import matplotlib.pyplot as plt
from lib.utils.visualize import visualize_merged
from PIL import Image
import numpy as np
import torch


def path_to_abblation(path):
    return "_".join(path.split("/")[-3].split("_")[1:])


def draw_and_save_color(dataset, idx, path):
    _path = f"/home/borth/GuidedResearch/data/dphm_kinect/{dataset}/color/{idx:05}.png"
    img = Image.open(_path)
    img.save(path)
    # plt.imshow(img)
    # plt.axis("off")  # Hide axes
    # plt.show()


def draw_and_save_weight(flame, renderer, out, path):
    renderer.update(1)
    mask = flame.render(renderer, out["params"])["mask"][0]
    renderer.update(8)

    # weight inference
    weights = out["optim_weights"][-1]
    weights = F.interpolate(
        weights.unsqueeze(0), scale_factor=8, mode="bilinear", align_corners=False
    )
    weights = weights.detach()[0][0]
    weights[~mask] = 0.0

    plt.figure(figsize=(19.2, 10.8), dpi=100)  # Full HD size
    plt.imshow(weights.cpu().numpy())
    plt.axis("off")  # Hide axes
    plt.savefig(path, bbox_inches="tight", pad_inches=0)  # Save without padding
    plt.show()


def draw_and_save_overlay(optimizer, renderer, params, dataset, idx, path):
    _path = f"/home/borth/GuidedResearch/data/dphm_kinect/{dataset}/color/{idx:05}.png"
    color = torch.tensor(np.asarray(Image.open(_path))).unsqueeze(0).to("cuda")
    renderer.update(scale=1)
    out = optimizer.flame.render(renderer, params)
    renderer.update(scale=8)
    img = visualize_merged(
        s_color=color,
        t_color=out["color"],
        t_mask=out["mask"],
    )
    img = img[0].detach().cpu().numpy()
    Image.fromarray(img).save(path)
    # plt.imshow(img)
    # plt.axis("off")  # Hide axes
    # plt.show()


def draw_and_save_normal(dataset, idx, path):
    _path = f"/home/borth/GuidedResearch/data/dphm_kinect/{dataset}/cache/2_normal/{idx:05}.pt"
    normal = torch.load(_path)
    _path = f"/home/borth/GuidedResearch/data/dphm_kinect/{dataset}/cache/2_mask/{idx:05}.pt"
    mask = torch.load(_path)
    normal_image = (((normal + 1) / 2) * 255).to(torch.uint8)
    normal_image[~mask] = 255
    Image.fromarray(normal_image.detach().cpu().numpy()).save(path)
    # plt.imshow(normal_image.detach().cpu().numpy())
    # plt.axis("off")  # Hide axes
    # plt.show()


def eval_iterations(
    optimizer,
    renderer,
    dataset,
    target_frame_idx,
    source_frame_idx,
    step_size=0.7,
    params=None,
    N=2,
):
    cfg = load_config("train", ["data=kinect"])
    datamodule = hydra.utils.instantiate(
        cfg.data,
        renderer=renderer,
        val_dataset=dict(
            start_frame=target_frame_idx,
            end_frame=target_frame_idx + 1,
            jump_size=target_frame_idx - source_frame_idx,
            datasets=[dataset],
        ),
    )
    datamodule.setup("fit")

    optimizer.max_iters = N
    optimizer.max_optims = 1
    optimizer.step_size = step_size
    out = None
    batch = None
    for i, b in enumerate(datamodule.val_dataloader()):
        with torch.no_grad():
            batch = optimizer.transfer_batch_to_device(b, "cuda", 0)
            if params is not None:
                batch["init_params"] = params
            out = optimizer(batch)
    return out, batch


def draw_and_save(img, path):
    # Display and save the error image
    Image.fromarray(img.detach().cpu().numpy()).save(path)
    # plt.imshow(img)
    # plt.axis("off")  # Hide axes
    # plt.show()


def load_flame_renderer():
    # instanciate similar to training
    cfg = load_config("train", ["data=kinect"])
    K = load_intrinsics(data_dir=cfg.data.intrinsics_dir, return_tensor="pt")
    camera = Camera(
        K=K,
        width=cfg.data.width,
        height=cfg.data.height,
        near=cfg.data.near,
        far=cfg.data.far,
        scale=cfg.data.scale,
    )
    rasterizer = Rasterizer(width=camera.width, height=camera.height)
    renderer = Renderer(rasterizer=rasterizer, camera=camera)
    flame = hydra.utils.instantiate(cfg.model)
    return flame, renderer


def render_output(renderer, optimizer, out, batch):
    renderer.update(scale=1)
    pred_out = optimizer.flame.render(renderer, out["params"])
    gt_out = optimizer.flame.render(renderer, batch["params"])
    error_map = visualize_point2plane_error(
        s_point=gt_out["point"][0],
        t_normal=pred_out["normal"][0],
        t_point=pred_out["point"][0],
        t_mask=pred_out["mask"][0],
        max_error=2e-03,  # 2mm
    )
    renderer.update(scale=8)
    color = pred_out["color"][0].detach().cpu()
    normal = pred_out["normal_image"][0].detach().cpu()
    return color, normal, error_map


def render(renderer, optimizer, out, batch):
    renderer.update(scale=1)
    pred_out = optimizer.flame.render(renderer, out["params"])
    gt_out = optimizer.flame.render(renderer, batch["params"])
    error_map = visualize_point2plane_error(
        s_point=gt_out["point"][0],
        t_normal=pred_out["normal"][0],
        t_point=pred_out["point"][0],
        t_mask=pred_out["mask"][0],
        max_error=6e-03,  # 2mm
    )
    renderer.update(scale=8)
    color = pred_out["color"][0].detach().cpu()
    return color, error_map


def load_neural_optimizer(flame, renderer, path, override=[]):
    cfg = load_config("train", ["data=kinect"] + override)
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    regularize = hydra.utils.instantiate(cfg.regularize)
    neural_optimizer = NeuralOptimizer.load_from_checkpoint(
        path,
        renderer=renderer,
        flame=flame,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        weighting=weighting,
    )
    return neural_optimizer


def load_icp_optimizer(flame, renderer, overrides):
    cfg = load_config("train", ["data=kinect", "optimizer.output_dir=none"] + overrides)
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    optimizer = hydra.utils.instantiate(cfg.optimizer)
    regularize = hydra.utils.instantiate(cfg.regularize)
    icp_optimizer = hydra.utils.instantiate(
        cfg.framework,
        flame=flame,
        logger=None,
        renderer=renderer,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        optimizer=optimizer,
        weighting=weighting,
    )
    return icp_optimizer.to("cuda")


# setup the datamodule
def load_datamodule(renderer, start_frame, end_frame, jump_size=1):
    cfg = load_config("train", ["data=kinect"])
    datamodule = hydra.utils.instantiate(
        cfg.data,
        renderer=renderer,
        val_dataset=dict(
            start_frame=start_frame, end_frame=end_frame, jump_size=jump_size
        ),
    )
    datamodule.setup("fit")
    return datamodule

In [ ]:
from tqdm.notebook import tqdm
from pathlib import Path
from lib.utils.video import create_video

# loadings
flame, renderer = load_flame_renderer()
setup = "presentation"

ours = "/home/borth/GuidedResearch/checkpoints/kinect/ours.ckpt"
neural_optimizer = load_neural_optimizer(flame, renderer, ours)

override = ["residuals=face2face_wo_landmarks", "regularize=dummy", "weighting=dummy"]
icp_optimizer = load_icp_optimizer(flame, renderer, override)

optimizers = [
    ("neural", neural_optimizer),
    ("icp", icp_optimizer),
]

settings = [
    ("ali_kocal_mouthmove", 10, 80),
    ("innocenzo_fulgintl_rotatemouth", 10, 80),
    ("christoph_mouthmove", 10, 80),
]

for (optimizer_name, optimizer) in optimizers:
    for (dataset, source_idx, target_idx) in settings:
        video_dir = f"results/{setup}/{dataset}/{optimizer_name}"
        params = None
        for idx in tqdm(range(source_idx, target_idx)):
            t_idx = idx + 1
            s_idx = idx
            path = ours
            out, batch = eval_iterations(
                optimizer,
                renderer,
                dataset,
                t_idx,
                s_idx,
                params=params,
            )
            params = out["params"]
            color, _, error = render_output(renderer, optimizer, out, batch)
            frame_path = f"{video_dir}/{t_idx:05}.png"
            Path(frame_path).parent.mkdir(exist_ok=True, parents=True)
            draw_and_save_overlay(optimizer, renderer, params, dataset, t_idx, frame_path)
        
            frame_path = f"results/{setup}/{dataset}/ground_truth/{t_idx:05}.png"
            Path(frame_path).parent.mkdir(exist_ok=True, parents=True)
            draw_and_save_overlay(optimizer, renderer, batch["params"], dataset, t_idx, frame_path) 
            frame_path = f"results/{setup}/{dataset}/color/{t_idx:05}.png"
            Path(frame_path).parent.mkdir(exist_ok=True, parents=True)
            draw_and_save_color(dataset, t_idx, frame_path)
            frame_path = f"results/{setup}/{dataset}/normal/{t_idx:05}.png"
            Path(frame_path).parent.mkdir(exist_ok=True, parents=True)
            draw_and_save_normal(dataset, t_idx, frame_path)

        video_path = Path(f"results/{setup}") / f"{dataset}_{optimizer_name}.mp4"
        create_video(video_dir=video_dir, video_path=video_path, framerate=16)

        video_path = Path(f"results/{setup}") / f"{dataset}_ground_truth.mp4"
        create_video(video_dir=f"results/{setup}/{dataset}/ground_truth", video_path=video_path, framerate=16)
        video_path = Path(f"results/{setup}") / f"{dataset}_color.mp4"
        create_video(video_dir=f"results/{setup}/{dataset}/color", video_path=video_path, framerate=16)
        video_path = Path(f"results/{setup}") / f"{dataset}_normal.mp4"
        create_video(video_dir=f"results/{setup}/{dataset}/normal", video_path=video_path, framerate=16)